In [2]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense


In [3]:
SF = pd.read_csv('SF_jobs_new.csv')
SF.drop('Unnamed: 0',axis=1,inplace=True)
NYC = pd.read_csv('NYC_jobs_new.csv')
NYC.drop('Unnamed: 0',axis=1,inplace=True)
Seattle = pd.read_csv('Seattle_jobs_new.csv')
Seattle.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
#Functions for data cleaning
wnl = WordNetLemmatizer()

def text_cleaner(text):
    
    text = text.lower().split()
    text = [wnl.lemmatize(myWord) for myWord in text] # Lemmertization

    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    
    return text # return list of tokens

In [4]:
cleanDescription = []
for i in range(SF.shape[0]):
    cleanDescription.append(text_cleaner(SF.job_description[i]))
for i in range(NYC.shape[0]):
    cleanDescription.append(text_cleaner(NYC.job_description[i]))
for i in range(Seattle.shape[0]):
    cleanDescription.append(text_cleaner(Seattle.job_description[i]))

In [7]:
len(cleanDescription)

1547

In [5]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=20,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [6]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [7]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
14     15.208069        1       1  0.042855  0.006409
9      10.439332        1       2  0.072920 -0.097090
2       9.188920        1       3  0.105061 -0.028974
16      9.083496        1       4  0.022591  0.040740
12      8.733611        1       5  0.069849 -0.081440
6       6.232983        1       6  0.026619 -0.020892
11      5.683452        1       7  0.074576 -0.000584
3       5.476119        1       8 -0.018934  0.005014
5       4.897409        1       9  0.052222 -0.056885
4       4.733617        1      10  0.122850  0.182932
8       4.571852        1      11 -0.045610 -0.037281
10      3.907470        1      12  0.044366  0.002748
13      3.363786        1      13 -0.067832 -0.043316
15      2.406697        1      14  0.067437  0.015017
1       2.059057        1      15 -0.014491  0.023508
7       1.231547        1      16 -0.100878  0.127271
19      0.921699        1      17 -0.104152 -0.007175
0       0.765121        1      18 -0.110903 -0.038732
17      0.655123        1      19 -0.132218  0.034730
18      0.440639        1      20 -0.106326 -0.026002, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
17402  Default   4234.000000     security   4234.000000  30.0000  30.0000
14331  Default  18273.000000         data  18273.000000  29.0000  29.0000
43698  Default   3409.000000      network   3409.000000  28.0000  28.0000
1759   Default  22006.000000   experience  22006.000000  27.0000  27.0000
9844   Default   1984.000000      analyst   1984.000000  26.0000  26.0000
2577   Default  11651.000000     business  11651.000000  25.0000  25.0000
29594  Default   9425.000000       system   9425.000000  24.0000  24.0000
8285   Default   2446.000000            u   2446.000000  23.0000  23.0000
4296   Default   6552.000000    knowledge   6552.000000  22.0000  22.0000
12738  Default   1306.000000     deloitte   1306.000000  21.0000  21.0000
4170   Default   3269.000000         test   3269.000000  20.0000  20.0000
27033  Default   7725.000000      project   7725.000000  19.0000  19.0000
5857   Default   6912.000000  application   6912.000000  18.0000  18.0000
315    Default   7436.000000      service   7436.000000  17.0000  17.0000
15144  Default   3952.000000  information   3952.000000  16.0000  16.0000
7150   Default   2816.000000      testing   2816.000000  15.0000  15.0000
32931  Default   7616.000000   management   7616.000000  14.0000  14.0000
2650   Default   3695.000000     customer   3695.000000  13.0000  13.0000
27423  Default   5901.000000      support   5901.000000  12.0000  12.0000
517    Default   4371.000000            .   4371.000000  11.0000  11.0000
19410  Default   2401.000000        cloud   2401.000000  10.0000  10.0000
41380  Default   1455.000000         sale   1455.000000   9.0000   9.0000
46553  Default   8211.000000        skill   8211.000000   8.0000   8.0000
1412   Default   8274.000000   technology   8274.000000   7.0000   7.0000
25150  Default   7215.000000       design   7215.000000   6.0000   6.0000
50097  Default   1908.000000         risk   1908.000000   5.0000   5.0000
3989   Default   5895.000000       client   5895.000000   4.0000   4.0000
14187  Default   9094.000000         team   9094.000000   3.0000   3.0000
37931  Default   2901.000000     engineer   2901.000000   2.0000   2.0000
43563  Default  10317.000000         work  10317.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
49554  Topic20     10.974828       puppet    169.387653   2.6881  -6.3440
4170   Topic20     30.254605         test   3269.055898   0.7421  -5.3300
315    Topic20     38.346068      service   7436.964472   0.1571  -5.0930
23250  Topic20     19.464651   deployment   1018.310785   1.4674  -5.7711
1759   Topic20     52.026417   